# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import pandas as pd
import numpy as np

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [7]:
# Your code here:
len(salaries[salaries.isnull().any(axis=1)])

33183

In [9]:
salaries.shape

(33183, 8)

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [10]:
# Your code here:
salaries["Salary or Hourly"].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [11]:
# Your code here:
salaries["Department"].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [25]:
# Your code here:
from scipy import stats
stats.ttest_1samp(salaries[salaries['Hourly Rate'].isnull() == False]['Hourly Rate'], 30)

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [29]:
# Your code here:
tstat, pvalue = stats.ttest_1samp(salaries[(salaries['Department'] == 'POLICE') & (salaries['Annual Salary'].isnull() == False)]['Annual Salary'], 86000)
print(tstat, pvalue/2)

3.081997005712994 0.0010301701775482569


Using the `crosstab` function, find the department that has the most hourly workers. 

In [40]:
# Your code here:
# first rows, then columnns
hourly_workers = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'], normalize='index')
hourly_workers

Salary or Hourly,Hourly,Salary
Department,,
ADMIN HEARNG,0.000000,1.000000
ANIMAL CONTRL,0.234568,0.765432
AVIATION,0.664211,0.335789
BOARD OF ELECTION,0.000000,1.000000
BOARD OF ETHICS,0.000000,1.000000
BUDGET & MGMT,0.043478,0.956522
BUILDINGS,0.000000,1.000000
BUSINESS AFFAIRS,0.040936,0.959064
CITY CLERK,0.000000,1.000000


In [36]:
hourly_workers[hourly_workers['Hourly'] == hourly_workers['Hourly'].max()]

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [54]:
# Your code here:
# workers STREETS & SAM: hourly wage < 35
# t-test, one-sided...95% confidence
stats.ttest_1samp(salaries[(salaries['Department'] == 'STREETS & SAN') & (salaries['Hourly Rate'].isnull() == False)]['Hourly Rate'], 35, alternative = 'less' )

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=1.6689265282353859e-21)

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [69]:
# Your code here:
# 95% CI for the mean hourly wage of all hourly workers
# t.interval, len - 1 degrees of freedom
# scipy.stats.sem --> standard error 
from scipy.stats import t
hourly = salaries.loc[(salaries['Hourly Rate'].isnull() == False), 'Hourly Rate']
mean_ = np.mean(hourly)
error_ = stats.sem(hourly)
stats.t.interval(.95, loc = mean_, scale = error_, df = len(hourly)-1)

(32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [62]:
# Your code here:
police = salaries.loc[(salaries['Department'] == "POLICE") & (salaries['Annual Salary'].isnull() == False), 'Annual Salary']
mean_ = np.mean(police)
error_ = stats.sem(police)
stats.t.interval(.95, loc = mean_, scale = error_, df = len(police)-1)

(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [75]:
# Your code here:
# We reject the null hypothesis, because the p-value is smaller than 0.05
from statsmodels.stats.proportion import proportions_ztest
hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']
lenght = len(salaries)
hourly_work = len(hourly)
proportions_ztest(hourly_work, lenght, value = .25)

(-3.5099964213703005, 0.0004481127249057967)